## Simulation Framework 

In [1]:
import import_ipynb
import math 
import importlib

# import functions from model_evaluation.ipynb  
import model_evaluation 

importing Jupyter notebook from model_evaluation.ipynb


In [2]:
# Define input variables  
group_dist = [[0], [1], [2], [3]] 
vote_dist = [5, 5, 5, 5]

In [3]:
# Assuming this is the correct function signature
result_group_memberships = model_evaluation.group_memberships(group_dist, vote_dist)
print(result_group_memberships)

[[0], [1], [2], [3]]


In [4]:
result_groups, result_aggregate_weight = model_evaluation.first_term(group_dist, vote_dist, result_group_memberships)
print(result_groups)
print(result_aggregate_weight)

{'group_0': {'agent_0': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}, 'group_1': {'agent_1': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}, 'group_2': {'agent_2': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}, 'group_3': {'agent_3': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}}
20.0


In [5]:
result_individual, result_aggregated = model_evaluation.interaction_terms(group_dist, result_group_memberships, vote_dist)
print(result_individual)
print(result_aggregated)

{'group_0': {'other_group_1': {'components_int_term1': {'agent_i_0': {'vote_attenuation_i': 5, 'num_groups_i': 1, 'vote_weight_i': 5.0}}, 'components_int_term2': {'agent_j_1': {'vote_attenuation_j': 5, 'num_groups_j': 1, 'vote_weight_j': 5.0}}, 'sum_vote_weight_i': 5.0, 'sqrt_sum_vote_weight_i': 2.23606797749979, 'sum_vote_weight_j': 5.0, 'sqrt_sum_vote_weight_j': 2.23606797749979, 'multiplied_interaction': 5.000000000000001}, 'other_group_2': {'components_int_term1': {'agent_i_0': {'vote_attenuation_i': 5, 'num_groups_i': 1, 'vote_weight_i': 5.0}}, 'components_int_term2': {'agent_j_2': {'vote_attenuation_j': 5, 'num_groups_j': 1, 'vote_weight_j': 5.0}}, 'sum_vote_weight_i': 5.0, 'sqrt_sum_vote_weight_i': 2.23606797749979, 'sum_vote_weight_j': 5.0, 'sqrt_sum_vote_weight_j': 2.23606797749979, 'multiplied_interaction': 5.000000000000001}, 'other_group_3': {'components_int_term1': {'agent_i_0': {'vote_attenuation_i': 5, 'num_groups_i': 1, 'vote_weight_i': 5.0}}, 'components_int_term2': {'

In [6]:
plurality_score = model_evaluation.connection_oriented_cluster_match(result_aggregate_weight, result_aggregated)
print(plurality_score)

8.94427190999916


In [7]:
result_quadratic_voting, result_sum_quadratic_votes = model_evaluation.quadratic_voting(vote_dist)
print(result_quadratic_voting)
print(result_sum_quadratic_votes)

{0: 2.23606797749979, 1: 2.23606797749979, 2: 2.23606797749979, 3: 2.23606797749979}
8.94427190999916
